In [11]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /var/folders/s6/3zpv69vn7939njdmq8qdwbrm0000gn/T/ipykernel_52440/3480413151.py:8 

In [12]:
# Read the csv files from datasets

#Fake news
df_fake = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/Fake.csv")
df_fake.show()

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|
| Papa John’s Foun...|A centerpiece of ...|   News|December 21, 2017|
| WATCH: Paul Ryan..

In [13]:
#Truth news
df_true = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/True.csv")
df_true.show()

+--------------------+--------------------+--------------------+------------------+
|               title|                text|             subject|              date|
+--------------------+--------------------+--------------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Factbox: Trump on...|The following sta...|        politicsNews|December 29, 2017 |
|Trump on Twitter ...|The following sta...|        politicsNews|December 29,

In [16]:
# count of data and the. umber of coloumns
print((df_fake.count(), len(df_fake.columns)))
print((df_true.count(), len(df_true.columns)))

(23489, 4)
(21417, 4)


In [26]:
#Add the column for fake news and also a column for true news
from pyspark.sql.functions import lit

df_fake = df_fake.withColumn("Flag", lit("Fake_News"))
df_fake.show()

+--------------------+--------------------+-------+-----------------+---------+
|               title|                text|subject|             date|     Flag|
+--------------------+--------------------+-------+-----------------+---------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_News|
| WATCH: Brand-New...|Just when you mig.

In [27]:
df_true = df_true.withColumn("Flag", lit("True_News"))
df_true.show()

+--------------------+--------------------+--------------------+------------------+---------+
|               title|                text|             subject|              date|     Flag|
+--------------------+--------------------+--------------------+------------------+---------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |True_News|
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |True_News|
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |True_News|
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Factbox: Trump on...|The following sta...|        politicsN

In [33]:
#concat the two tables